In [137]:
# use matplotlib to visualize the downsampled stack
%matplotlib qt
# get the stack and downsample it
from deeptrace import *
from deeptrace.plotting import *

atlas,ontology,header = read_atlas()
params,autof,models = load_deeptrace_models('d:/WholeBrain/sampledata/deeptrace_analysis/210723_NAc326F_488_s3_0_8x_13-31-25')
def reduce_ontology(ontology,atlas):
    uatlas = np.unique(atlas)
    nont = []
    for i,o in enumerate(ontology.atlas_id.values):
        if o in uatlas:
            nont.append(ontology.iloc[i])
    return pd.DataFrame(nont)
# ontology = reduce_ontology(ontology,atlas)


In [107]:
ontology


,id,atlas_id,name,acronym,st_level,ontology_id,hemisphere_id,weight,parent_structure_id,depth,...,structure_id_path,color_hex_triplet,neuro_name_structure_id,neuro_name_structure_id_path,failed,sphinx_id,structure_name_facet,failed_facet,safe_name,main_parent
1,8,0.0,Basic cell groups and regions,grey,1,1,3,8690,997.0,1,...,/997/8/,BFDAE3,NaN,NaN,f,2,2244697386,734881840,Basic cell groups and regions,r
5,315,746.0,Isocortex,Isocortex,5,1,3,8690,695.0,5,...,/997/8/567/688/695/315/,70FF71,NaN,NaN,f,6,2323732626,734881840,Isocortex,Isoctx
15,299,1027.0,"Somatomotor areas, Layer 5",MO5,11,1,3,8690,500.0,7,...,/997/8/567/688/695/315/500/299/,1F9D5A,NaN,NaN,f,16,851674589,734881840,Somatomotor areas Layer 5,Isoctx
28,1021,1117.0,"Secondary motor area, layer 6a",MOs6a,11,1,3,8690,993.0,8,...,/997/8/567/688/695/315/500/993/1021/,1F9D5A,NaN,NaN,f,29,3489757563,734881840,Secondary motor area layer 6a,Isoctx
30,453,339.0,Somatosensory areas,SS,6,1,3,8690,315.0,6,...,/997/8/567/688/695/315/453/,188064,NaN,NaN,f,31,3016301862,734881840,Somatosensory areas,Isoctx
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1302,153,726.0,lateral recess,V4r,9,1,3,8690,145.0,3,...,/997/73/145/153/,AAAAAA,NaN,NaN,f,1303,4006455201,734881840,lateral recess,r
1304,1024,693.0,grooves,grv,1,1,3,8690,997.0,1,...,/997/1024/,AAAAAA,NaN,NaN,f,1305,1816833424,734881840,grooves,r
1307,1063,698.0,hippocampal fissure,hf,8,1,3,8690,1032.0,3,...,/997/1024/1032/1063/,AAAAAA,NaN,NaN,f,1308,1845828307,734881840,hippocampal fissure,r
1313,1103,703.0,primary fissure,pri,8,1,3,8690,1040.0,3,...,/997/1024/1040/1103/,AAAAAA,NaN,NaN,f,1314,1359711822,734881840,primary fissure,r


In [160]:
# get all IDs for structure
def get_atlas_mask(ontology, atlas, acronym,uatlas = None):
    if uatlas is None:
        uatlas = np.unique(atlas)
    st_level = ontology[ontology.acronym == acronym].st_level.iloc[0]
    parent_id = '/' + str(ontology[ontology.acronym == acronym].id.iloc[0])+'/'
    idpaths = ontology['structure_id_path'].values
    maxdepth = ontology.depth.max()
    ids = []
    for i,o in ontology.iterrows():
        if parent_id in o.structure_id_path:
            if np.isfinite(o.id):
                ids.append(int(o.id))
    mask = np.zeros_like(atlas,dtype = bool)
    areas = []
    for i in ids:
        if i in uatlas:
            mask[atlas == i] = True 
            areas.append(ontology[ontology.id == i])
    return mask,pd.concat(areas)
mask,areas = get_atlas_mask(ontology,atlas,'Isocortex')
areas

,id,atlas_id,name,acronym,st_level,ontology_id,hemisphere_id,weight,parent_structure_id,depth,...,structure_id_path,color_hex_triplet,neuro_name_structure_id,neuro_name_structure_id_path,failed,sphinx_id,structure_name_facet,failed_facet,safe_name,main_parent
6,184,871.0,"Frontal pole, cerebral cortex",FRP,8,1,3,8690,315.0,6,...,/997/8/567/688/695/315/184/,268F45,NaN,NaN,f,7,2565719060,734881840,Frontal pole cerebral cortex,Isoctx
18,985,830.0,Primary motor area,MOp,8,1,3,8690,500.0,7,...,/997/8/567/688/695/315/500/985/,1F9D5A,NaN,NaN,f,19,1852742012,734881840,Primary motor area,Isoctx
24,993,831.0,Secondary motor area,MOs,8,1,3,8690,500.0,7,...,/997/8/567/688/695/315/500/993/,1F9D5A,NaN,NaN,f,25,1043755260,734881840,Secondary motor area,Isoctx
37,322,747.0,Primary somatosensory area,SSp,8,1,3,8690,453.0,7,...,/997/8/567/688/695/315/453/322/,188064,NaN,NaN,f,38,131561364,734881840,Primary somatosensory area,Isoctx
100,378,754.0,Supplemental somatosensory area,SSs,8,1,3,8690,453.0,7,...,/997/8/567/688/695/315/453/378/,188064,NaN,NaN,f,101,713142416,734881840,Supplemental somatosensory area,Isoctx
107,1057,131.0,Gustatory areas,GU,6,1,3,8690,315.0,6,...,/997/8/567/688/695/315/1057/,009C75,NaN,NaN,f,108,722362724,734881840,Gustatory areas,Isoctx
114,677,367.0,Visceral area,VISC,8,1,3,8690,315.0,6,...,/997/8/567/688/695/315/677/,11AD83,NaN,NaN,f,115,3549914160,734881840,Visceral area,Isoctx
122,1011,833.0,Dorsal auditory area,AUDd,8,1,3,8690,247.0,7,...,/997/8/567/688/695/315/247/1011/,19399,NaN,NaN,f,123,2633724239,734881840,Dorsal auditory area,Isoctx
136,1002,832.0,Primary auditory area,AUDp,8,1,3,8690,247.0,7,...,/997/8/567/688/695/315/247/1002/,19399,NaN,NaN,f,137,760301534,734881840,Primary auditory area,Isoctx
143,1027,835.0,Posterior auditory area,AUDpo,8,1,3,8690,247.0,7,...,/997/8/567/688/695/315/247/1027/,19399,NaN,NaN,f,144,57661303,734881840,Posterior auditory area,Isoctx


In [154]:
ontology[ontology.atlas_id==8]

,id,atlas_id,name,acronym,st_level,ontology_id,hemisphere_id,weight,parent_structure_id,depth,...,structure_id_path,color_hex_triplet,neuro_name_structure_id,neuro_name_structure_id_path,failed,sphinx_id,structure_name_facet,failed_facet,safe_name,main_parent
735,72,8.0,Anterodorsal preoptic nucleus,ADP,8,1,3,8690,141.0,6,...,/997/8/343/1129/1097/141/72/,FF5547,NaN,NaN,f,736,1069999127,734881840,Anterodorsal preoptic nucleus,HY


In [161]:
interact_show_stack(mask,clim=[0,1])


In [136]:
masks = ['MO5','MOs6a']
for m in masks:
    mask = get_atlas_mask(ontology,atlas,m)
    

AttributeError: 'list' object has no attribute 'shape'

145
